# Assignment 2: Task 2

โดย

ตุลธร วงศ์ชัย รหัสนักศึกษา 63070224

ธนภัทร ธีรรัตตัญญู รหัสนักศึกษา 63070227

> **README**:
> - 'Run all' is not recommended. 
> - Section 1 must be run before any other section.
> - Read and follow instruction of each section carefully if it is provided.


# 1. Init Project

> **Instruction**: 
> - Set file path of `traffy_fondue_tickets.csv` in section 1.2 before running this section.
> - Although `attacut` is not imported directly, it is required installation.

## 1.1 Import Libraries and Packages

In [1]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [130]:
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [136]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, hamming_loss

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

## 1.2 Import Dataset

In [2]:
raw_data = pd.read_csv(f'{os.getcwd()}/traffy_fondue_tickets.csv')
raw_data

,running_id,issues,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,ความสะอาด,จราจร,ต้นไม้,...,ป้ายจราจร,ร้องเรียน,สอบถาม,สะพาน,สัตว์จรจัด,สายไฟ,ห้องน้ำ,เสนอแนะ,เสียงรบกวน,แสงสว่าง
0,1,ทำเรื่องรับสมุดทหารกองหนุน หลังจบแล้วมีให้เสีย...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2,ท่อประปาแตกหน้าบ้าน บ้านเลขที่126 ซ.สุขสวัสดิ์...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,3,ขอแจ้งเหตุถนนเป็นหลุม ที่บริเวณหน้าหมู่บ้าน ca...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,โครงการเพอร์เฟคเพลสพระราม9กรุงเทพกรีฑา ใช้รถน้...,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,5,เคยแจ้งทาง เขตแล้วให้ช่วยเก็บตามบ้าน แต่ละซอย ...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8584,10105,น้ำท่วมขังสูง มาตั้งแต่วันจันทร์ที่ 1 สค จนถึง...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8585,10106,สะพานลอยแถวถนนกัลปพฤกษ์ก่อนถึงแยกราชพฤกษ์ ทำไม...,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
8586,10107,มีผู้ค้ารายใหม่มาตั้งร้านบนถนนในจุดที่เจ้าหน้า...,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
8587,10108,1.ปัญหาการจราจรในซอยบางแค-บางบอน แยกพัฒนาการ เ...,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


# 2. Text Preprocessing

## 2.1 Define Word Tokenizers

2.1.1 กำหนดเซ็ตของ stop words ตามเซลล์โค้ดที่ 2.1.1

In [4]:
### Code Cell 2.1.1 ###

STOP_WORDS = thai_stopwords()

2.1.2 สร้าง tokenizer โดยนำเอา word tokenizer จาก PyThaiNLP มาเพิ่มการกรองให้เหลือแต่คำที่มีตัวอักษรภาษาไทยและภาษาอังกฤษโดยใช้ regular expression และกรองเอาคำ stop words ออก ตามเซลล์โค้ดที่ 2.1.2

In [7]:
### Code Cell 2.1.2 ###

def thai_word_tokenizer(text: str, engine: str) -> list:
    """
    Tokenize given text using PyThaiNLP's word tokenizer
    with filtering stop words and matching only word containing 
    Thai letters and English letters.

    Parameters
    ----------
    text : str
      Text to be tokenized
    engine : str
      Name of engine available in PyThaiNLP
      e.g. newmm, longest, attacut, deepcut 

    Returns
    ----------
    list
      List of tokens
    """
    tokens = word_tokenize(text, engine=engine)

    tokens_to_return = list()
    for token in tokens:
        if (re.match(r"([\u0E00-\u0E7Fa-zA-Z])+", token) and (token not in STOP_WORDS)):
            tokens_to_return.append(token)
    return tokens_to_return


2.1.3 สร้าง tokenizer จากตัว thai_word_tokenizer โดยกำหนด engine ดังนี้

- newmm ที่เป็น dictionary-based
- attacut ที่เป็น learning-based

ตามเซลล์โค้ดที่ 2.1.3

In [8]:
### Code Cell 2.1.3 ###

def newmm_tokenizer(text:str) -> list:
    """
    Tokenize given text using custom Thai word tokenizer
    with newmm engine

    Parameters
    ----------
    text : str
      Text to be tokenized

    Returns
    ----------
    list
      List of tokens
    """
    return thai_word_tokenizer(text, engine='newmm')

def attacut_tokenizer(text:str) -> list:
    """
    Tokenize given text using custom Thai word tokenizer
    with attacut engine

    Parameters
    ----------
    text : str
      Text to be tokenized

    Returns
    ----------
    list
      List of tokens
    """
    return thai_word_tokenizer(text, engine='attacut')

2.1.4 ทดสอบ tokenizer ที่สร้างขึ้นในข้อ 2.1.3 ตามเซลล์โค้ดที่ 2.1.4

In [20]:
text = """สำหรับวัคซีนไฟเซอร์ฝาส้ม เป็นวัคซีนสูตรสำหรับเด็ก (paediatric formulation dose) 
ซึ่งต่างจากวัคซีนสูตรผู้ใหญ่ที่เป็นฝาสีม่วง ข้อมูล ณ วันที่ 21 ม.ค. กรมควบคุมโรคระบุว่า 
ได้รับแจ้งจากบริษัทผู้ผลิตว่าส่งมอบได้ในไตรมาสแรกปีนี้ 3.5 ล้านโดส 
ทั้งนี้ ครม.อนุมัติงบประมาณจัดซื้อไว้ 10 ล้านโดส"""

print('Original Text:')
print(text)
print('Tokenized by newmm tokenizer:')
print(newmm_tokenizer(text))
print('Tokenized by attacut tokenizer:')
print(attacut_tokenizer(text))


Original Text:
สำหรับวัคซีนไฟเซอร์ฝาส้ม เป็นวัคซีนสูตรสำหรับเด็ก (paediatric formulation dose) 
ซึ่งต่างจากวัคซีนสูตรผู้ใหญ่ที่เป็นฝาสีม่วง ข้อมูล ณ วันที่ 21 ม.ค. กรมควบคุมโรคระบุว่า 
ได้รับแจ้งจากบริษัทผู้ผลิตว่าส่งมอบได้ในไตรมาสแรกปีนี้ 3.5 ล้านโดส 
ทั้งนี้ ครม.อนุมัติงบประมาณจัดซื้อไว้ 10 ล้านโดส
Tokenized by newmm tokenizer:
['สำหรับ', 'วัคซีน', 'ไฟ', 'เซอร์', 'ฝา', 'ส้ม', 'วัคซีน', 'สูตร', 'สำหรับ', 'เด็ก', 'paediatric', 'formulation', 'dose', 'วัคซีน', 'สูตร', 'ผู้ใหญ่', 'ฝา', 'สีม่วง', 'ข้อมูล', 'วันที่', 'ม.ค.', 'กรม', 'ควบคุม', 'โรค', 'ระบุ', 'แจ้ง', 'บริษัทผู้ผลิต', 'ส่งมอบ', 'ไตรมาส', 'ปี', 'ล้าน', 'โดส', 'ครม.', 'อนุมัติ', 'งบประมาณ', 'จัดซื้อ', 'ล้าน', 'โดส']
Tokenized by attacut tokenizer:
['สำหรับ', 'วัคซีน', 'ไฟเซอร์', 'ฝา', 'ส้ม', 'วัคซีน', 'สูตร', 'สำหรับ', 'เด็ก', 'paediatric', 'formulation', 'dose', 'วัคซีน', 'สูตร', 'ฝา', 'สี', 'ม่วง', 'ข้อมูล', 'ม.ค.', 'กรมควบคุมโรค', 'ระบุ', 'แจ้ง', 'บริษัท', 'ผลิต', 'ส่งมอบ', 'ไตรมาส', 'ปี', 'ล้าน', 'โดส', 'ครม.', 'อนุมัติ', 'ง

## 2.2 Vectorizing Docs

2.2.1 ทำการ tokenize ข้อความด้วย tokenizer จากข้อ 2.1 จากนั้นนำไปเข้า TFIDF vectorizer เพื่อทำการ term weighting โดยที่กำหนด min_df อยู่ที่ 30 และ max_df อยู่ที่ 0.95 ตามเซลล์โค้ดที่ 2.2.1

In [91]:
docs = raw_data['issues']
X_tfidf = dict()

tokenizers = [newmm_tokenizer, attacut_tokenizer]

for tokenizer in tokenizers:
    print(f'vectorizing docs with {tokenizer.__name__}')
    vectorizer = TfidfVectorizer(tokenizer=tokenizer, min_df=30, max_df=.95)
    term_weighted = vectorizer.fit_transform(docs)
    X_tfidf[tokenizer.__name__] = {
        'data': term_weighted.toarray(),
        'vectorizer': vectorizer
    }


vectorizing docs with newmm_tokenizer
vectorizing docs with attacut_tokenizer


# 3. Classification 1

นำข้อมูลที่ได้เตรียมไว้จากข้อ 2 (X_tfidf) มาทำการทดลองกับโมเดล KNN และ Random Forest

3.1 กำหนด model และช่วงของค่า parameter ที่ต้องการทำ tuning ตามเซลล์โค้ดที่ 3.1

In [42]:
### Code Cell 3.1 ###

models = [
    {
        "model": KNeighborsClassifier(),
        "params": {"n_neighbors": list(range(3, 16, 2))}
    },
    {
        "model": RandomForestClassifier(random_state=28),
        "params": {
            "n_estimators": list(range(25, 101, 25)),
            "min_samples_split": list(range(5, 26, 5)),
        }
    },
]

3.2 ทำการ parameter tuning หา parameter ที่ให้ค่า accuracy ที่ดีที่สุดด้วย 5-fold cross validation (GridSearchCV) ของคู่ model กับข้อมูลที่ผ่านการเตรียมด้วย tokenizer แบบต่างๆ จากนั้นนำมาทดสอบกับ test set เพื่อประเมินประสิทธิภาพ model ตามเซลล์โค้ดที่ 3.2

In [43]:
### Code Cell 3.2 ###

results1 = list()

y = raw_data.iloc[:, 2:]

for tokenizer_name, X in X_tfidf.items():
    print(f'Docs tokenized by {tokenizer_name}')

    # Train-test split with test size of 33%
    X_train, X_test, y_train, y_test = train_test_split(X['data'], y,
        random_state=50, test_size=.33)

    for model in models:
        print(f"- Tuning {type(model['model']).__name__}:")

        # Search for best set of parameters, using 5-fold cross validation
        clf = GridSearchCV(
            model['model'], model['params'], cv=5, n_jobs=-1, verbose=1)
        clf.fit(X_train, y_train)
        print(
            f'Best tuning: acc {clf.best_score_:.4f}, params {clf.best_params_}')

        # Test model with best set of parameters from GridSearchCV
        y_pred = clf.best_estimator_.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_pred)
        test_hamming_loss = hamming_loss(y_test, y_pred)
        print(f'Testing accuracy: {test_accuracy:.4f}, ' +
              f'hamming loss: {test_hamming_loss:.4f}')

        result = {
            'model': type(model['model']).__name__,
            'tokenizer': tokenizer_name,
            'best_tuning_params': clf.best_params_,
            'best_tuning_acc': clf.best_score_,
            'best_estimator': clf.best_estimator_,
            'accuracy': test_accuracy,
            'hamming_loss': test_hamming_loss}

        results1.append(result)

    print('-'*10)


Docs tokenized by newmm_tokenizer
- Tuning KNeighborsClassifier:
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best tuning: acc 0.1945, params {'n_neighbors': 13}
Testing accuracy: 0.1954, hamming loss: 0.0580
- Tuning RandomForestClassifier:
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best tuning: acc 0.3116, params {'min_samples_split': 5, 'n_estimators': 100}
Testing accuracy: 0.3266, hamming loss: 0.0461
----------
Docs tokenized by attacut_tokenizer
- Tuning KNeighborsClassifier:
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best tuning: acc 0.2452, params {'n_neighbors': 15}
Testing accuracy: 0.2519, hamming loss: 0.0516
- Tuning RandomForestClassifier:
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best tuning: acc 0.3222, params {'min_samples_split': 5, 'n_estimators': 75}
Testing accuracy: 0.3362, hamming loss: 0.0445
----------


3.3 แสดงผลลัพธ์จากข้อ 3.2 ตามเซลล์โค้ดที่ 3.3.1 - 3.3.2

In [44]:
### Code Cell 3.3.1 ###

result_df = pd.DataFrame(results1)

result_df[['model', 'tokenizer', 'accuracy', 'hamming_loss']] \
    .sort_values(by=['accuracy', 'hamming_loss'], ascending=False) \
    .reset_index(drop=True)


,model,tokenizer,accuracy,hamming_loss
0,RandomForestClassifier,attacut_tokenizer,0.336155,0.044490
1,RandomForestClassifier,newmm_tokenizer,0.326631,0.046101
2,KNeighborsClassifier,attacut_tokenizer,0.251852,0.051591
3,KNeighborsClassifier,newmm_tokenizer,0.195414,0.058002


จากผลลัพธ์ของเซลล์โค้ดที่ 3.3 พบว่าผลการทดลองที่ดีที่สุด คือ การเตรียมข้อมูลด้วย attacut_tokenizer และใช้ RandomForestClassifier ที่กำหนดให้ n_estimators = 75 และ min_samples_split = 5 ซึ่งมีค่า accuracy = 0.336155 และ hamming loss = 0.044490

In [55]:
### Code Cell 3.3.2 ###

# Best model
results1[3]

{'model': 'RandomForestClassifier',
 'tokenizer': 'attacut_tokenizer',
 'best_tuning_params': {'min_samples_split': 5, 'n_estimators': 75},
 'best_tuning_acc': 0.3222128206096778,
 'best_estimator': RandomForestClassifier(min_samples_split=5, n_estimators=75, random_state=28),
 'accuracy': 0.3361552028218695,
 'hamming_loss': 0.04449045318610536}

# 4. Classification 2 (proposed idea)

นำ model ที่ดีที่สุดจากการทดลองในข้อ 3 (RandomForestClassifier ที่กำหนดให้ n_estimators = 75 และ min_samples_split = 5) มาปรับเพิ่ม prop ของผลลัพธ์ของแต่ละ label class อิงจาก frequent_terms ของ class นั้นๆ

กำหนดให้ frequent_term คือ term ที่มี document frequency สูง โดยเทียบกับ document ที่อยู่ใน label class หนึ่งๆ

4.1 หา frequent_term ของแต่ละ label class โดยกำหนดให้ term ที่จะเป็น frequent_term ได้ต้องมี df >= 0.75

In [92]:
### Code Cell 4.1 ###

frequent_terms_in_classes = list()

for i in range(23):
    print(f'{raw_data.columns[2+i]}', end=': ')
    docs = raw_data['issues'][raw_data.iloc[:,2+i]==1]

    # Find frequent terms
    try:
      count_vectorizer = CountVectorizer(tokenizer=attacut_tokenizer, 
          token_pattern=None, min_df=.75)
      count_vectorizer.fit(docs)
      frequent_terms = list(count_vectorizer.vocabulary_.keys())
    except:
      frequent_terms = list()
    
    print(frequent_terms)
    frequent_terms_in_classes.append(frequent_terms)
    

การเดินทาง: []
กีดขวาง: []
คนจรจัด: ['คน']
คลอง: ['คลอง']
ความปลอดภัย: []
ความสะอาด: []
จราจร: ['รถ']
ต้นไม้: ['ต้น', 'ไม้']
ถนน: ['ถนน']
ทางเท้า: []
ท่อระบายน้ำ: ['ท่อ', 'น้ำ']
น้ำท่วม: ['ท่วม', 'น้ำ']
ป้าย: ['ป้าย']
ป้ายจราจร: ['ป้าย']
ร้องเรียน: []
สอบถาม: []
สะพาน: ['สะพาน']
สัตว์จรจัด: []
สายไฟ: ['สาย']
ห้องน้ำ: ['ห้องน้ำ']
เสนอแนะ: []
เสียงรบกวน: ['เสียง']
แสงสว่าง: ['ไฟ']


จากผลลัพธ์เซลล์โค้ดที่ 4.1 แสดงผลลัพธ์ของ frequent_terms ของแต่ละ label class เช่น frequent_terms ของ label class 'คนจรจัด' ได้แก่ ['คน']; frequent_terms ของ label class 'ท่อระบายน้ำ' ได้แก่ ['ท่อ', 'น้ำ']

4.2 แปลง term เป็น index ของ term ใน feature vector ตามเซลล์โค้ดที่ 4.2

In [97]:
### Code Cell 4.2 ###

vectorizer = X_tfidf['attacut_tokenizer']['vectorizer']

frequent_terms_in_classes_index = list()

for label_class in frequent_terms_in_classes:
    frequent_terms_index = list()
    for frequent_term in label_class:
        frequent_terms_index.append(vectorizer.vocabulary_[frequent_term])
    frequent_terms_in_classes_index.append(frequent_terms_index)

4.3 สร้างฟังก์ชันในการปรับ prop โดยจะปรับ prop ของ label class ขึ้น 0.2 หาก document มี frequent term ของ label class นั้น ตามเซลล์โค้ดที่ 4.3

In [128]:
### Code Cell 4.3 ###

def adjust_label_class_prop(X, prop, label_idx:int):
    adjusted_prop = [prop[0], prop[1]]
    for frequent_term_idx in frequent_terms_in_classes_index[label_idx]:
        
        # Check if doc has a frequent term
        if (X[frequent_term_idx] > 0):
            adjusted_prop[1] += 0.2
            break
    return adjusted_prop

4.4 สร้างฟังก์ชันในการใช้ทำนายผลลัพธ์จากการปรับ prob ของแต่ละ label class ตามเซลล์โค้ดที่ 4.4

In [133]:
### Code Cell 4.4 ###

def predict_proposed_idea(X):
    clf = results1[3]['best_estimator']
    pred_prob = clf.predict_proba(X)
    
    y_pred = list()
    for i in range(len(X)):
        pred = list()
        for label_idx in range(23):
            prop = pred_prob[label_idx][i]
            adjusted_prop = adjust_label_class_prop(X[i], prop, label_idx)
            pred_label_class = 1 if adjusted_prop[1] > adjusted_prop[0] else 0
            pred.append(pred_label_class)
        y_pred.append(pred)
    return y_pred

4.5 ทำการทดสอบโมเดลโดยใช้การปรับ prop ด้วย frequent_terms ตามเซลล์โค้ดที่ 4.5

In [134]:
### Code Cell 4.5 ###

# Train-test split with test size of 33%
_, X_test, _, y_test = train_test_split(X_tfidf['attacut_tokenizer']['data'], y,
    random_state=50, test_size=.33)

y_pred = predict_proposed_idea(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_hamming_loss = hamming_loss(y_test, y_pred)
print(f'Testing accuracy: {test_accuracy:.6f}, ' +
      f'hamming loss: {test_hamming_loss:.6f}')

Testing accuracy: 0.344974, hamming loss: 0.044107


จากผลลัพธ์ของเซลล์โค้ดที่ 4.5 พบว่าโมเดลมีค่า accuracy เพิ่มขึ้นจาก 0.336155 เป็น 0.344974 และค่า hamming loss ลดลงจาก 0.044490 เป็น 0.044107

# 5. Classification 3

นำข้อมูลที่ได้เตรียมไว้จากข้อ 2 (X_tfidf) มาทำการทดลองกับโมเดล OneVsRestClassifier โดยใช้ LogisticRegression

5.1 กำหนด model และช่วงของค่า parameter ที่ต้องการทำ tuning ตามเซลล์โค้ดที่ 5.1

In [151]:
### Code Cell 5.1 ###

models = [
    {
        "model": LogisticRegression(solver='lbfgs'),
        "params": {
            "estimator__C": np.arange(0.2, 1.1, 0.2),
        }
    },
]

5.2 ทำการ parameter tuning หา parameter ที่ให้ค่า accuracy ที่ดีที่สุดด้วย 5-fold cross validation (GridSearchCV) ของคู่ model กับข้อมูลที่ผ่านการเตรียมด้วย tokenizer แบบต่างๆ จากนั้นนำมาทดสอบกับ test set เพื่อประเมินประสิทธิภาพ model ตามเซลล์โค้ดที่ 3.2

In [152]:
### Code Cell 5.2 ###

results2 = list()

y = raw_data.iloc[:, 2:]

for tokenizer_name, X in X_tfidf.items():
    print(f'Docs tokenized by {tokenizer_name}')

    # Train-test split with test size of 33%
    X_train, X_test, y_train, y_test = train_test_split(X['data'], y,
        random_state=50, test_size=.33)

    for model in models:
        print(f"- Tuning OVR using {type(model['model']).__name__}:")

        # Search for best set of parameters, using 5-fold cross validation
        ovr_model = OneVsRestClassifier(model['model'])
        clf = GridSearchCV(
            ovr_model, model['params'], cv=5, n_jobs=-1, verbose=1)
        clf.fit(X_train, y_train)
        print(
            f'Best tuning: acc {clf.best_score_:.4f}, params {clf.best_params_}')

        # Test model with best set of parameters from GridSearchCV
        y_pred = clf.best_estimator_.predict(X_test)
        test_accuracy = accuracy_score(y_test, y_pred)
        test_hamming_loss = hamming_loss(y_test, y_pred)
        print(f'Testing accuracy: {test_accuracy:.4f}, ' +
              f'hamming loss: {test_hamming_loss:.4f}')

        result = {
            'model': type(model['model']).__name__,
            'tokenizer': tokenizer_name,
            'best_tuning_params': clf.best_params_,
            'best_tuning_acc': clf.best_score_,
            'best_estimator': clf.best_estimator_,
            'accuracy': test_accuracy,
            'hamming_loss': test_hamming_loss}

        results2.append(result)

    print('-'*10)


Docs tokenized by newmm_tokenizer
- Tuning OVR using LogisticRegression:
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best tuning: acc 0.2760, params {'estimator__C': 1.0}
Testing accuracy: 0.2864, hamming loss: 0.0474
----------
Docs tokenized by attacut_tokenizer
- Tuning OVR using LogisticRegression:
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best tuning: acc 0.2963, params {'estimator__C': 1.0}
Testing accuracy: 0.3178, hamming loss: 0.0449
----------


จากผลลัพธ์เซลล์โค้ดที่ 5.2 พบว่าผลการทดลองที่ดีที่สุด คือ การเตรียมข้อมูลด้วย attacut_tokenizer และใช้ OneVsRestClassifier กับ LogisticRegression ที่กำหนดค่า C = 1.0 ซึ่งมีค่า accuracy = 0.3178 และ hamming loss = 0.0449

# Summary

จากการทดลองทั้งหมด การจำแนก ticket โดยการเตรียมข้อมูลด้วย attacut_tokenizer และใช้โมเดล RandomForest ให้ผลลัพธ์ที่ดีที่สุด โดยมีค่า accuracy = 0.336155 และ hamming loss = 0.044490 และเมื่อใช้ proposed idea ในการปรับผลลัพธ์ของโมเดล ทำให้โมเดลมีค่า accuracy = 0.344974 (เพิ่มขึ้นมา 0.008819)